In [55]:
import os
import glob
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from datetime import date
from datetime import datetime
import calendar
from openpyxl.utils import get_column_letter
from datetime import datetime, timedelta
import xlrd
from openpyxl.utils.exceptions import InvalidFileException

In [264]:
# Diretório de downloads
diretorio_downloads = os.path.join(os.path.expanduser('~'), 'Downloads')

# Diretório de documentos
diretorio_documentos = os.path.join(os.path.expanduser('~'), 'Documents')

# Padrão de nome do arquivo
padrao_nome_arquivo = '*.xlsx'

# Caminho completo para o arquivo de saída na pasta de downloads
arquivo_saida = os.path.join(diretorio_documentos, 'resumo_semanal_modificado.xlsx')

# Listar todos os arquivos que correspondem ao padrão de nome
arquivos = glob.glob(os.path.join(diretorio_downloads, padrao_nome_arquivo))

# Verificar se há arquivos correspondentes
if arquivos:
    # Selecionar o último arquivo pela data de modificação
    arquivo_mais_recente = max(arquivos, key=os.path.getmtime)
    
    # Carregar o arquivo Excel
    workbook = pd.ExcelFile(arquivo_mais_recente)
    
    # Criar um escritor Excel
    writer = pd.ExcelWriter(arquivo_saida, engine='xlsxwriter')
    
    # Ler o conteúdo da primeira aba para obter a data
    df_primeira_aba = workbook.parse(workbook.sheet_names[0], header=None)
    cabecalho_index = df_primeira_aba[df_primeira_aba.iloc[:, 0] == 'DATA INICIAL'].index[0]
    data_atual = df_primeira_aba.iloc[cabecalho_index + 1, 0].strftime('%d/%m/%Y')  # Manter a data como está no arquivo original
    data_anterior = (pd.to_datetime(data_atual, format='%d/%m/%Y') - pd.DateOffset(days=7)).strftime('%d/%m/%Y')
    
    # Imprimir as datas
    print(f'Data atual: {data_atual}')
    print(f'Data anterior: {data_anterior}')
    
    # Iterar sobre as abas (tabelas)
    for nome_aba in workbook.sheet_names:
        # Ler o conteúdo da aba em um DataFrame
        df_temporario = workbook.parse(nome_aba, header=None)  # Não usar cabeçalho
        cabecalho_index = df_temporario[df_temporario.iloc[:, 0] == 'DATA INICIAL'].index[0]
        df_temporario.columns = df_temporario.iloc[cabecalho_index]  # Usar a linha correta como cabeçalho
        df_temporario = df_temporario.iloc[cabecalho_index+1:]  # Excluir linhas anteriores ao cabeçalho
        
        # Modificar as datas para o formato desejado '%d/%m/%Y'
        df_temporario['DATA INICIAL'] = pd.to_datetime(df_temporario['DATA INICIAL']).dt.strftime('%d/%m/%Y')
        df_temporario['DATA FINAL'] = pd.to_datetime(df_temporario['DATA FINAL']).dt.strftime('%d/%m/%Y')
        
        # Salvar o DataFrame como uma aba no arquivo Excel
        df_temporario.to_excel(writer, sheet_name=nome_aba, index=False, na_rep='')

    
    # Fechar o escritor Excel
    writer.save()
    
    print(f'Arquivo salvo com sucesso em {arquivo_saida}')
else:
    print("Nenhum arquivo encontrado com o padrão de nome especificado.")

Data atual: 09/04/2023
Data anterior: 02/04/2023
Arquivo salvo com sucesso em C:\Users\ricardo\Documents\resumo_semanal_modificado.xlsx


In [265]:
# Importar a biblioteca openpyxl
from openpyxl import load_workbook

# Caminho do arquivo Excel
caminho_resumo_modificado = r'C:\Users\ricardo\Documents\resumo_semanal_modificado.xlsx'

# Carregar o arquivo Excel
workbook_resumo = load_workbook(filename=caminho_resumo_modificado, read_only=True, data_only=True)

# Caminho do arquivo Excel
caminho_arquivo = r'Z:\IPTC\13 - Analista de Dados_Bases\01 - Painel do Diesel - Bases\Ricardo\base_diesel_desde__31_07_2022.xlsx'

# Carregar o arquivo Excel
workbook_base = load_workbook(filename=caminho_arquivo, read_only=True, data_only=True)

# Listar todas as abas
abas = workbook_base.sheetnames

# Fechar o arquivo Excel
workbook_base.close()

# Mostrar as abas
print("Abas presentes no arquivo:")
for aba in abas:
    print(aba)
    
    
# Caminho da planilha base
caminho_planilha_base = r'Z:\IPTC\13 - Analista de Dados_Bases\01 - Painel do Diesel - Bases\Ricardo\base_diesel_desde__31_07_2022.xlsx'

# Caminho do arquivo resumo_semanal_modificado
caminho_resumo_modificado = r'C:\Users\ricardo\Documents\resumo_semanal_modificado.xlsx'

# Carregar a planilha base
workbook_base = load_workbook(filename=caminho_planilha_base, data_only=True)

# Lista de abas a serem transferidas
abas = ['CAPITAIS', 'MUNICIPIOS', 'ESTADOS', 'REGIOES', 'BRASIL']

# Carregar o arquivo resumo_semanal_modificado
df_resumo_modificado = pd.read_excel(caminho_resumo_modificado)

# Carregar o arquivo resumo_semanal_modificado
xls = pd.ExcelFile(caminho_resumo_modificado)

# Obter a data atual
data_atual = df_resumo_modificado.iloc[0]['DATA INICIAL']

# Para cada aba a ser transferida
for aba in abas:
    # Verificar se a aba está presente no arquivo modificado
    if aba in xls.sheet_names:
        # Carregar os dados da aba correspondente no arquivo modificado
        df_aba_modificada = pd.read_excel(caminho_resumo_modificado, sheet_name=aba)
        
        # Carregar a aba correspondente na planilha base
        aba_base = workbook_base[aba]
        
        # Verificar se a data atual já está presente na coluna "DATA INICIAL"
        datas_na_aba = [aba_base.cell(row=row, column=1).value for row in range(2, aba_base.max_row + 1)]
        if data_atual in datas_na_aba:
            print(f'Dados já atualizados na aba {aba}')
        else:
            # Encontrar a primeira célula vazia na coluna A
            idx = 1
            while aba_base.cell(row=idx, column=1).value is not None:
                idx += 1
            
            # Preencher os valores na coluna correspondente na planilha base
            for i, row in df_aba_modificada.iterrows():
                for j, value in enumerate(row):
                    aba_base.cell(row=idx, column=j+1, value=value)
                idx += 1
                
# Salvar as alterações na planilha base como um arquivo XLSX
workbook_base.save(caminho_planilha_base)

print(f'Alterações salvas na planilha base como {caminho_planilha_base}')


Abas presentes no arquivo:
Panorama geral de qtd pesquisas
New Gas Database
Capitais-
Serie Historica def
New Gas Database - SP ANO
PROC REGIOES-ESTADOS
CAPITAIS
MUNICIPIOS
ESTADOS
REGIOES
BRASIL
Alterações salvas na planilha base como Z:\IPTC\13 - Analista de Dados_Bases\01 - Painel do Diesel - Bases\Ricardo\base_diesel_desde__31_07_2022.xlsx


In [266]:
workbook.close()
workbook_base.close()
workbook_resumo.close()